# Importing Libraries

In [1]:
import numpy as np
import cv2
import skimage.transform
import imutils
import time

from imutils.video import VideoStream
from matplotlib import pyplot as plt
from keras.models import load_model

Using TensorFlow backend.
C:\Users\Prateek\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Prateek\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Prateek\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Prateek\anaconda3\lib\site-packages\tensorflow\python\framework\d

# Defining Mask Detection Function

In [3]:
def detect_and_predict_mask(Frame, FaceNet, MaskNet):
    
    faces = []
    face_count = 0
    locs = []
    prediction = []
    
    (h, w) = Frame.shape[:2]
    
    # Making a Blob of Dimension (300x300)
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),(104.0, 177.0, 123.0))
    
    # Detecting Faces via Caffe Model
    FaceNet.setInput(blob)
    detections = FaceNet.forward()
    
    for i in range(0, detections.shape[2]):
        
        confidence = detections[0, 0, i, 2]
        # In Detections 0,0,i,2 is the confidence while 0,0,i,3:7 are location of the box
        
        # If confidence of the detected object of being a face is above 0.5 then only it is taken into consideration
        if confidence > 0.5:
            
            # Getting the Box location from the Detection
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")
            
            # If Box exceed the Image Boundaries
            (startX, startY) = (max(0, startX), max(0, startY))
            (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
            
            ## PRE-PROCESSING THE FACES ##
            
            # Croping only the face which will be feeded to the model
            face = Frame[startY:endY, startX:endX]
            # Converting Image to Grayscale as Mask-Detection model is trained on it
            face = np.dot(face,[0.2989, 0.5870, 0.1140])
            # Resizing Gray Image ---> (200x200)
            face = skimage.transform.resize(face, (200, 200))
            # Scaling Gray-Values Between 0 and 1
            face = face/255
            # Re-Dimensioning to 4D array that will be feeded to the CNN
            face = np.expand_dims(face, -3)
            face = np.expand_dims(face, -1)
            
            face_count =+ 1
            faces.append(face)
            locs.append((startX, startY, endX, endY))
            
    for i in range(face_count):
        
        # Predicting if the person is wearing a mask or not
        pred = MaskNet.predict(faces[i])
        prediction.append(pred)
        
    # Returning the Location of the Box and Prediction
    return (locs, prediction)

# Initializing Face-Detection and Mask-Detection Network

In [4]:
prototxtPath = ("deploy.prototxt")
weightsPath = ("res10_300x300_ssd_iter_140000.caffemodel")

# Loading Caffe-Model for Face Detection
FaceNet = cv2.dnn.readNet(prototxtPath,weightsPath)

# Loading Mask-Detection-Model
MaskNet = load_model('Model')

C:\Users\Prateek\anaconda3\lib\site-packages\keras\engine\saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


# Initializing Video-Stream

In [5]:
vs = VideoStream(src=0).start()
time.sleep(0.5)

# Looping over Frames in Stream and Detecting Faces With or Without Mask

In [6]:
while True:
    
    frame = vs.read()
    frame = imutils.resize(frame, width=720)
    
    (locs, preds) = detect_and_predict_mask(frame, FaceNet, MaskNet)
    
    
    ## DRAWING BOXES AROUND DETECTED FACES ##
    
    for (box, pred) in zip(locs, preds):
        
        (startX, startY, endX, endY) = box
        (mask, withoutMask) = pred[0]
        
        label = "MASK" if mask > withoutMask else "NO MASK"
        color = (0, 255, 0) if label == "MASK" else (0, 0, 255)
        
        label = "{}: {:.2f}%".format(label, max(mask, withoutMask) * 100)
        
        cv2.putText(frame, label, (startX, startY - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 0.75, color, 2)
        cv2.rectangle(frame, (startX, startY), (endX, endY), color, 2)
        
    cv2.imshow("Video-Stream", frame)
    
    # Closing the Video Stream when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
            
cv2.destroyAllWindows()
vs.stop()

KeyboardInterrupt: 